In [1]:
%cd ..

c:\Users\habanoz\Desktop\gpt\nb_gpu_trainer


In [2]:
from tokenizer.tokenizer import Tokenizer
from huggingface_hub import login
from trainer import TrainerConfig, Trainer
from hf_backed_trainer import HFBackedTrainer
from model.gpt2 import GPT

In [3]:
login()

In [4]:
trainer_cfg_file = "config/news_trainer.yml"
model_cfg_file = "config/news_model.yml"

In [5]:
def load_config(config_file)->TrainerConfig:
    import yaml

    with open(config_file) as f:
        try:
            doc = yaml.safe_load(f)
        except yaml.YAMLError as exc:
            print(exc)
            exit(1)
    
    return TrainerConfig(**doc)

trainer_cfg = load_config(trainer_cfg_file)

In [6]:
trainer_cfg.repo_id, trainer_cfg.ds_repo_id

('habanoz/haber-gpt-v1.0', 'habanoz/eco-news-tr')

## Prepare Data

In [7]:
tokenizer = Tokenizer.from_pretrained(trainer_cfg.repo_id)

tokenizer.model:   0%|          | 0.00/383k [00:00<?, ?B/s]

c:\Users\habanoz\miniconda3\envs\myenv\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\habanoz\.cache\huggingface\hub\models--habanoz--haber-gpt-v1.0. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [8]:
tokenizer.encode_ds_from_hub(trainer_cfg.ds_repo_id, trainer_cfg.data_dir)

Number of 'train' tokens: 4767677
Number of 'validation' tokens: 535192


## Train the model

In [9]:
model = GPT.from_config(model_cfg_file)
model.to("cuda")

number of parameters: 13.77M


GPT(
  (transformer): ModuleDict(
    (wte): Embedding(8192, 384)
    (wpe): Embedding(512, 384)
    (drop): Dropout(p=0.2, inplace=False)
    (h): ModuleList(
      (0-5): 6 x Block(
        (ln_1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=384, out_features=1152, bias=False)
          (c_proj): Linear(in_features=384, out_features=384, bias=False)
          (attn_dropout): Dropout(p=0.2, inplace=False)
          (resid_dropout): Dropout(p=0.2, inplace=False)
        )
        (ln_2): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Linear(in_features=384, out_features=1536, bias=False)
          (gelu): GELU(approximate='none')
          (c_proj): Linear(in_features=1536, out_features=384, bias=False)
          (dropout): Dropout(p=0.2, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head)

In [10]:
trainer = HFBackedTrainer.from_config(trainer_cfg_file)
trainer.train(model)

pytorch_model.bin:   0%|          | 0.00/55.9M [00:00<?, ?B/s]

trainer_state.bin:   0%|          | 0.00/112M [00:00<?, ?B/s]

Resume training. Using repo habanoz/haber-gpt-v1.0
Loading model state dict!
Loaded model state dict!


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: huseyinabanozis. Use `wandb login --relogin` to force relogin


num decayed parameter tensors: 26, with 13,959,168 parameters
num non-decayed parameter tensors: 13, with 4,992 parameters


...

iter 750: loss 2.9291, run_iter_time 951.00ms, fb_toks/sec 17228.10, run_fb_toks/sec 17228.10
Completed haber-gpt\pytorch_model.bin
iter 760: loss 2.6150, run_iter_time 953.81ms, fb_toks/sec 16734.78, run_fb_toks/sec 17178.77
iter 770: loss 2.8479, run_iter_time 961.78ms, fb_toks/sec 15852.74, run_fb_toks/sec 17046.17
iter 780: loss 2.7325, run_iter_time 977.10ms, fb_toks/sec 14694.13, run_fb_toks/sec 16810.96
iter 790: loss 2.6428, run_iter_time 987.65ms, fb_toks/sec 15134.49, run_fb_toks/sec 16643.31
Completed haber-gpt\trainer_state.bin
iter 800: loss 2.6024, run_iter_time 996.09ms, fb_toks/sec 15282.67, run_fb_toks/sec 16507.25
iter 810: loss 2.7854, run_iter_time 990.08ms, fb_toks/sec 17504.89, run_fb_toks/sec 16607.01
iter 820: loss 2.5581, run_iter_time 1005.37ms, fb_toks/sec 14334.04, run_fb_toks/sec 16379.72
iter 830: loss 2.5355, run_iter_time 1022.34ms, fb_toks/sec 13943.27, run_fb_toks/sec 16136.07
iter 840: loss 2.7172, run_iter_time 1037.16ms, fb_toks/sec 13996.58, run_fb

pytorch_model.bin:   0%|          | 0.00/55.9M [00:00<?, ?B/s]

trainer_state.bin:   0%|          | 0.00/112M [00:00<?, ?B/s]

iter 1000: loss 2.5153, run_iter_time 1095.57ms, fb_toks/sec 14050.99, run_fb_toks/sec 15016.69
iter 1010: loss 2.5902, run_iter_time 1122.01ms, fb_toks/sec 12047.04, run_fb_toks/sec 14719.72
iter 1020: loss 2.6978, run_iter_time 1105.71ms, fb_toks/sec 17085.04, run_fb_toks/sec 14956.26
iter 1030: loss 2.6432, run_iter_time 1142.34ms, fb_toks/sec 11130.63, run_fb_toks/sec 14573.69
Completed haber-gpt\pytorch_model.bin
iter 1040: loss 2.6090, run_iter_time 1142.30ms, fb_toks/sec 14346.67, run_fb_toks/sec 14550.99
iter 1050: loss 2.5420, run_iter_time 1141.97ms, fb_toks/sec 14384.98, run_fb_toks/sec 14534.39
Completed haber-gpt\trainer_state.bin
iter 1060: loss 2.6675, run_iter_time 1144.03ms, fb_toks/sec 14092.93, run_fb_toks/sec 14490.24


KeyboardInterrupt: 